## 3 EKF SLAM

在上一节中我们看到的是扩展卡尔曼滤波在定位中的应用，EKF同样可以应用于SLAM问题中。在定位问题中，机器人接收到的观测值是其在二维空间中的x-y位置。如果机器人接收到的是跟周围环境有关的信息，例如机器人某时刻下距离某路标点的距离和角度，那么我们可以根据此时机器人自身位置的估计值，推测出该路标点在二维空间中的位置，将路标点的空间位置也作为待修正的状态量放入整个的状态向量中。

由此又引出了两个子问题：数据关联和状态扩增。

我们将通过以下python实例来理解ekf slam的原理：

![ekf_slam](img/ekf_slam.png)

- 黑色星号: 路标点真实位置（代码中RFID数组代表真实位置）
- 绿色×: 路标点位置估计值
- 黑色线: 航迹推测方法得出的机器人轨迹 
- 蓝色线: 轨迹真值
- 红色线: EKF SLAM得出的机器人轨迹

回想上一节的扩展卡尔曼滤波的公式：

=== Predict ===

$x_{Pred} = Fx_t+Bu_t$ 

$P_{Pred} = J_FP_t J_F^T + Q$

=== Update ===

$z_{Pred} = Hx_{Pred}$ 

$y = z - z_{Pred}$

$S = J_H P_{Pred}.J_H^T + R$

$K = P_{Pred}.J_H^T S^{-1}$

$x_{t+1} = x_{Pred} + Ky$

$P_{t+1} = ( I - K J_H) P_{Pred}$

EKF SLAM 使用一个EKF滤波器来解决SLAM问题，EKF SLAM的状态向量包括了机器人位姿 $(x, y, \theta)$ 和观测到的各个路标点的坐标 $[(m_1x, m_1y), (m_2x, m_2y), ... , (m_nx, m_ny)]$ ，路标点与机器人位姿间的协方差矩阵也在更新。

![ekfslam_state_vector](img/state_vec.jpg)

相应的协方差矩阵：

![ekfslam_cov](img/state_cov.jpg)

可以简写作：

![ekfslam_cov_simple](img/state_cov_s.jpg)

需要注意的是，由于状态向量中包含路标点位置坐标，所以随着机器人运动，观测到的路标点会越来越多，因此状态向量和状态协方差矩阵会不断变化。

### 运动模型

**状态向量预测** 在状态向量中，只有机器人位姿状态$(x, y, \theta)$是随着机器人运动改变的，所以运动模型影响下的状态向量预测这步中只改变状态向量中的这三个值和协方差矩阵中对应的区域。以下是本实例中使用的运动模型，控制输入向量为机器人线速度和角速度 $(v,w)$。

$\begin{equation*}
F=
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 
\end{bmatrix}
\end{equation*}$

$\begin{equation*}
B=
\begin{bmatrix}
\Delta t cos(\theta) & 0\\
\Delta t sin(\theta) & 0\\
0 & \Delta t
\end{bmatrix}
\end{equation*}$

$\begin{equation*}
U=
\begin{bmatrix}
v_t\\
w_t\\
\end{bmatrix}
\end{equation*}$

$\begin{equation*}
X = FX + BU 
\end{equation*}$


$\begin{equation*}
\begin{bmatrix}
x_{t+1} \\
y_{t+1} \\
\theta_{t+1}
\end{bmatrix}=
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 
\end{bmatrix}\begin{bmatrix}
x_{t} \\
y_{t} \\
\theta_{t}
\end{bmatrix}+
\begin{bmatrix}
\Delta t cos(\theta) & 0\\
\Delta t sin(\theta) & 0\\
0 & \Delta t
\end{bmatrix}
\begin{bmatrix}
v_{t} + \sigma_v\\
w_{t} + \sigma_w\\
\end{bmatrix}
\end{equation*}$

可以看出运动模型与上一节中的并无不同（只是状态向量有区别），$U$ 是包含线速度角速度 $v_t$ 和 $w_t$的控制输入向量，$+\sigma_v$ 和 $+\sigma_w$ 表示控制输入也包含噪声。

**协方差预测** 对机器人位姿状态的协方差进行预测，本质上是加上运动模型误差，位姿的不确定性通过加上运动模型的协方差$Q$而增加了。

$
P = G^TPG + Q
$

注意：这里的$G$就是上一节公式中的雅克比矩阵$J_F$，为与python代码保持一致，这里不再改动。

在这一步骤中，协方差矩阵$P$里只有与机器人位姿相关的部分被修改，与路标点相关的部分没有改动。

In [ ]:
def motion_model(x, u):
    """
    根据控制预测机器人位姿
    """
    F = np.array([[1.0, 0, 0],
                  [0, 1.0, 0],
                  [0, 0, 1.0]])

    B = np.array([[DT * math.cos(x[2, 0]), 0],
                  [DT * math.sin(x[2, 0]), 0],
                  [0.0, DT]])

    x = (F @ x) + (B @ u)
    # 返回3x1
    return x

# 计算雅克比矩阵，与上一节相比并无不同，只不过因为机器人位姿状态向量少了一个元素，jF是3×3：
def jacob_motion(x, u):
    Fx = np.hstack((np.eye(STATE_SIZE), np.zeros(
        (STATE_SIZE, LM_SIZE * calc_n_lm(x)))))

    jF = np.array([[0.0, 0.0, -DT * u[0] * math.sin(x[2, 0])],
                   [0.0, 0.0, DT * u[0] * math.cos(x[2, 0])],
                   [0.0, 0.0, 0.0]], dtype=np.float64)

    G = np.eye(STATE_SIZE) + Fx.T @ jF @ Fx
    # 返回雅克比矩阵G(3x3)，和对角矩阵Fx(实际上的np.eye(3))
    return G, Fx,

### 观测模型

在这个实例中，观测模型比上一节要复杂，因为实例模拟的是激光雷达检测路标点获得的与机器人距离和角度的信息。观测值向量$z$包含的两个元素就是机器人与路标点的相对距离和角度，因此单个路标点在二维空间内的坐标就可以通过机器人在二维空间内的坐标得出：

![obs1](img/obs1.jpg)

等式左边就是第j个路标点在二维空间内的坐标。

相应地，可以得到机器人位姿状态与单个路标点观测值之间的转换关系，也就是观测模型：

![obs2](img/obs2.jpg)

所以单个路标点的雅克比矩阵表示为：

![obs3](img/obs3.jpg)

（说明：$H_t$矩阵左上角的low表示这是针对单个路标点的雅克比矩阵，右上角的i表示这是对应第i组观测值的雅克比矩阵）

对于每个路标点，观测雅克比矩阵需要进行偏微分的有以下5个元素：

$(\overline\mu_{t,x}, \overline\mu_{t,y}, \overline\mu_{t,\theta}, \overline\mu_{j,x},\overline\mu_{j,y})$

分别是机器人位姿状态和该路标点的二维x-y空间位置。

推导可得：

![obs5](img/obs5.jpg)

![obs6](img/obs6.jpg)

单个路标点的雅克比矩阵作用于整个观测模型雅克比矩阵上需要乘以一个矩阵$F_{x,j}$，来把单个雅克比矩阵的元素放到对应位置上，j就表示是第几个路标点。

![obs7](img/obs7.jpg)

$2j-2=2(j-1)$表示第j个路标点前的j-1个路标点的位置，$2N-2j=2(N-j)$表示第j个路标点之后的N-j个路标点（N在此表示一共有N个路标点）的位置，这些位置都是0，因为该路标点不会作用于其他路标点。

In [ ]:
def calc_innovation(lm, xEst, PEst, z, LMid):
    """
    Calculates the innovation based on expected position and landmark position
    
    :param lm:   landmark position
    :param xEst: estimated position/state
    :param PEst: estimated covariance
    :param z:    read measurements
    :param LMid: landmark id
    :returns:    returns the innovation y, and the jacobian H, and S, used to calculate the Kalman Gain
    """
    delta = lm - xEst[0:2]
    q = (delta.T @ delta)[0, 0]
    z_angle = math.atan2(delta[1, 0], delta[0, 0]) - xEst[2, 0]
    zp = np.array([[math.sqrt(q), pi_2_pi(z_angle)]]) # 对应以上的观测模型 
    y = (z - zp).T
    y[1] = pi_2_pi(y[1])
    # 放入delta的平方（标量）,delta(2x1),状态向量和lm在状态向量中的序号+1
    H = jacob_h(q, delta, xEst, LMid + 1)
    S = H @ PEst @ H.T + R

    return y, S, H

def jacob_h(q, delta, x, i):
    """
    Calculates the jacobian of the measurement function
    
    :param q:     the range from the system pose to the landmark
    :param delta: the difference between a landmark position and the estimated system position
    :param x:     the state, including the estimated system position
    :param i:     landmark id + 1
    :returns:     the jacobian H
    """
    sq = math.sqrt(q)
    G = np.array([[-sq * delta[0, 0], - sq * delta[1, 0], 0, sq * delta[0, 0], sq * delta[1, 0]],
                  [delta[1, 0], - delta[0, 0], - q, - delta[1, 0], delta[0, 0]]]) # 单个路标点的雅克比矩阵
    G = G / q # 单个路标点的雅克比矩阵
    nLM = calc_n_lm(x) # 路标点总数
    F1 = np.hstack((np.eye(3), np.zeros((3, 2 * nLM))))

    F2 = np.hstack((np.zeros((2, 3)), np.zeros((2, 2 * (i - 1))),
                    np.eye(2), np.zeros((2, 2 * nLM - 2 * i))))
    F = np.vstack((F1, F2))
    H = G @ F

    return H

其中jacob_h函数中的这几行代码：

In [ ]:
F1 = np.hstack((np.eye(3), np.zeros((3, 2 * nLM))))
F2 = np.hstack((np.zeros((2, 3)), np.zeros((2, 2 * (i - 1))),
                    np.eye(2), np.zeros((2, 2 * nLM - 2 * i))))
F = np.vstack((F1, F2))

就是在构建上面推导的算式中的$F_{x,j}$矩阵，在代码中，nLM表示路标点总数，i表示路标点的id，运行几个例子来看看$F_{x,j}$矩阵的细节：

In [4]:
import numpy as np
nLM = 3 # 假设共有三个路标点
i = 1
F1 = np.hstack((np.eye(3), np.zeros((3, 2 * nLM))))
F2 = np.hstack((np.zeros((2, 3)), np.zeros((2, 2 * (i - 1))),np.eye(2), np.zeros((2, 2 * nLM - 2 * i))))
F = np.vstack((F1, F2))
print(F1)
print(F2)
print(F)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.]]


In [5]:
i = 2
F1 = np.hstack((np.eye(3), np.zeros((3, 2 * nLM))))
F2 = np.hstack((np.zeros((2, 3)), np.zeros((2, 2 * (i - 1))),np.eye(2), np.zeros((2, 2 * nLM - 2 * i))))
F = np.vstack((F1, F2))
print(F1)
print(F2)
print(F)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]]


In [6]:
i = 3
F1 = np.hstack((np.eye(3), np.zeros((3, 2 * nLM))))
F2 = np.hstack((np.zeros((2, 3)), np.zeros((2, 2 * (i - 1))),np.eye(2), np.zeros((2, 2 * nLM - 2 * i))))
F = np.vstack((F1, F2))
print(F1)
print(F2)
print(F)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]


在实例中，机器人在每个时刻t都会得到若干组观测值，就像在现实中，每隔一定时间，机器人都会得到传感器的一系列测量值一样。那么如何知道一组观测值$z_t^i=(r_t^i,\phi_t^i)^T$ 对应的是第几个路标点呢？也就是说，上面第j个路标点的j是怎么得到的？下面简述本python实例中的方案：

对于每一组观测值$z_t^i=(r_t^i,\phi_t^i)^T$，计算对应此时刻的机器人位姿，已有的状态向量：

![state_vec_1](img/state_vec.jpg)

中的从landmark 1到landmark n的每一个路标点的观测值，也就是从此时刻机器人的位置向四周看去，landmark 1到landmark n的观测值，计算这些观测值与观测值$z_t^i=(r_t^i,\phi_t^i)^T$的**残差**，如果这些残差的**马哈拉诺比斯距离**均大于某一个阈值，则判定这是一个没观测过的新的路标点；如果这些**距离**不是都大于某个阈值，则我们简单地认为**距离**最短的那个路标点landmark就是观测值$z_t^i=(r_t^i,\phi_t^i)^T$对应的路标点。简单来说，找跟观测值$z_t^i=(r_t^i,\phi_t^i)^T$差别最小的那个路标点，如果差别都挺大，那就认为看到了个新的路标点。

**马哈拉诺比斯距离**的阈值在代码中的值为2.0:

In [7]:
M_DIST_TH = 2.0  # Threshold of Mahalanobis distance for data association.
## 注：马哈拉诺比斯距离Mahalanobis distance是由印度统计学家马哈拉诺比斯提出的，表示数据的协方差距离。
## 它是一种有效的计算两个未知样本集的相似度的方法。与欧氏距离不同的是它考虑到各种特性之间的联系并且是尺度无关的，
## 即独立于测量尺度（来自wiki）。

在实例中寻找匹配的路标点编号id的对应代码是：

In [ ]:
def search_correspond_landmark_id(xAug, PAug, zi):
    """
    Landmark association with Mahalanobis distance
    """
    # 此时已有的LM的个数：
    nLM = calc_n_lm(xAug)

    min_dist = []

    for i in range(nLM):
        # 返回第i个已有LM的坐标lm：
        lm = get_landmark_position_from_state(xAug, i)
        # 放入第i个已有LM的坐标lm、机器人位姿预测过的状态向量xAug和variance PAug、待比较的新观测zi
        y, S, H = calc_innovation(lm, xAug, PAug, zi, i)
        # 由残差y计算Mahalanobis distance：y.T @ np.linalg.inv(S) @ y
        min_dist.append(y.T @ np.linalg.inv(S) @ y)

    min_dist.append(M_DIST_TH)  # new landmark

    min_id = min_dist.index(min(min_dist))

    return min_id

这种根据残差的马哈拉诺比斯距离最小来寻找匹配的路标点只是一种理想化的简单处理方式，实际应用中的“data association”会复杂得多。

接下来还有一步需要注意，如果$z_t^i=(r_t^i,\phi_t^i)^T$是一个未观测过的路标点的观测值，那么需要对状态向量和状态协方差矩阵进行扩增，对应的python代码为：

In [ ]:
# Extend state and covariance matrix
xAug = np.vstack((xEst, calc_landmark_position(xEst, z[iz, :])))
PAug = np.vstack((np.hstack((PEst, np.zeros((len(xEst), LM_SIZE)))),
                  np.hstack((np.zeros((LM_SIZE, len(xEst))), initP))))
xEst = xAug
PEst = PAug

其中的initP是一个对角元素为1的2×2矩阵，代表给新增的一个路标点的状态量协方差一个初始值。

至此，我们了解了机器人对路标点的观测模型，以及如何计算观测模型的雅克比矩阵，接下来就可以继续计算卡尔曼增益$K$，执行EKF中的**更新**步骤了。整个**更新**步骤的算法总结如下：

![obs8](img/obs8.jpg)
![obs9](img/obs9.jpg)

实例中整个EKF_SLAM算法的框架如下，具体各个函数的实现详见代码文件：

In [ ]:
def ekf_slam(xEst, PEst, u, z):
    # Predict
    # 预测过程只与运动模型有关
    S = STATE_SIZE
    # 返回运动模型雅克比G和对角矩阵Fx
    G, Fx = jacob_motion(xEst[0:S], u)
    # 返回机器人位姿的在含噪控制下的预测：
    xEst[0:S] = motion_model(xEst[0:S], u)
    # 更新机器人位姿的P
    PEst[0:S, 0:S] = G.T @ PEst[0:S, 0:S] @ G + Fx.T @ Q @ Fx
    # 给Landmark准备的variance
    initP = np.eye(2) 

    # Update
    # 对在此时刻新得到的一组z含噪观测，对z中的每一个元素做操作：
    for iz in range(len(z[:, 0])):  # for each observation
        min_id = search_correspond_landmark_id(xEst, PEst, z[iz, 0:2])

        nLM = calc_n_lm(xEst)
        # 在函数search_correspond_landmark_id中，把M_DIST_TH放在了最后一个，
        # 又因为已有的LM的序号值范围是（0至（nLM-1）），所以最后一个值M_DIST_TH的序号就是nLM，
        # 所以如果min_id等于nLM，意味着新观测z[iz, 0:2]与所有已有观测的马哈拉诺比斯距离都超过了M_DIST_TH，
        # 即这是一个以前没看到过的LM：
        if min_id == nLM:
            print("New LM")
            # Extend state and covariance matrix
            xAug = np.vstack((xEst, calc_landmark_position(xEst, z[iz, :])))
            PAug = np.vstack((np.hstack((PEst, np.zeros((len(xEst), LM_SIZE)))),
                              np.hstack((np.zeros((LM_SIZE, len(xEst))), initP))))
            xEst = xAug
            PEst = PAug
        # 如果min_id不等于nLM，则认为这是一个已经观测过的LM，马哈拉诺比斯距离最小的那个，也就是min_id指向的那个就是对应的已观测LM：
        lm = get_landmark_position_from_state(xEst, min_id)
        y, S, H = calc_innovation(lm, xEst, PEst, z[iz, 0:2], min_id)

        K = (PEst @ H.T) @ np.linalg.inv(S)
        xEst = xEst + (K @ y)
        PEst = (np.eye(len(xEst)) - (K @ H)) @ PEst

    xEst[2] = pi_2_pi(xEst[2])

    return xEst, PEst

最后再说一点此实例中有关仿真数据生成的内容，在observation函数中，根据机器人位姿真值决定机器人可以看到哪几个路标点，再将这几个路标点的观测值人为加上噪声。